In [1]:
!rm -r data
import os
try:
  os.mkdir('./data')
except:
  pass
!wget https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt -P ./data

rm: cannot remove 'data': No such file or directory
--2020-09-27 08:30:03--  https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘./data/input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2020-09-27 08:30:03 (7.55 MB/s) - ‘./data/input.txt’ saved [1115394/1115394]



In [2]:
!pip install unidecode
import torch
import torch.nn as nn
import unidecode
import string
import random
import re
import time,math

     |████████████████████████████████| 245kB 2.8MB/s 


In [3]:
num_epochs = 2000
print_every = 100
plot_every = 10
chunk_len = 200
hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

In [4]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


In [5]:
file = unidecode.unidecode(open('./data/input.txt').read())
file_len = len(file)
print('file_len = ',file_len)

file_len =  1115394


In [6]:
def random_chunk():
  start_index = random.randint(0,file_len - chunk_len)
  end_index = start_index + chunk_len +1 
  return file[start_index:end_index]
print(random_chunk())

 for me
To help thee curse that poisonous bunchback'd toad.

HASTINGS:
False-boding woman, end thy frantic curse,
Lest to thy harm thou move our patience.

QUEEN MARGARET:
Foul shame upon you! you have


In [7]:
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
    tensor[c] = all_characters.index(string[c])
  return tensor
print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


In [8]:
def random_training_set():
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp,target

In [9]:
class RNN(nn.Module):
  def __init__(self,input_size,embedding_size,hidden_size,output_size,num_layers = 1):
    super(RNN,self).__init__()
    self.input_size = input_size
    self.embedding_size = embedding_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers

    self.encoder = nn.Embedding(self.input_size,self.embedding_size)
    self.rnn = nn.LSTM(self.embedding_size,self.hidden_size,self.num_layers)
    self.decoder = nn.Linear(self.hidden_size,self.output_size)

  def forward(self,input,hidden,cell):
    out = self.encoder(input.view(1,-1))
    out,(hidden,cell) = self.rnn(out,(hidden,cell))
    out = self.decoder(out.view(batch_size,-1))
    return out,hidden,cell
  
  def init_hidden(self):
    hidden = torch.zeros(self.num_layers,batch_size,self.hidden_size)
    cell = torch.zeros(self.num_layers,batch_size,self.hidden_size)
    return hidden,cell
model = RNN(n_characters,embedding_size,hidden_size,n_characters,num_layers)

In [10]:
inp = char_tensor('A')
print(inp)
hidden,cell = model.init_hidden()
print(hidden.size())
out,hidden,cell = model(inp,hidden,cell)
print(out.size())

tensor([36])
torch.Size([1, 1, 100])
torch.Size([1, 100])


In [11]:
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
loss_func = nn.CrossEntropyLoss()

In [14]:
def test():
  start_str = 'b'
  inp = char_tensor(start_str)
  hidden,cell = model.init_hidden()
  x = inp

  print(start_str,end ='')
  for i in range(200):
    output,hidden,cell = model(x,hidden,cell)
    output_dist = output.data.view(-1).div(0.8).exp()
    top_i = torch.multinomial(output_dist,1)[0]
    predicted_char = all_characters[top_i]

    print(predicted_char,end='')
    x= char_tensor(predicted_char)

In [15]:
for i in range(num_epochs):
  inp,label = random_training_set()
  hidden,cell = model.init_hidden()

  loss = torch.tensor([0]).type(torch.FloatTensor)
  optimizer.zero_grad()
  for j in range(chunk_len-1):
    x = inp[j]
    y_ = label[j].unsqueeze(0).type(torch.LongTensor)
    y,hidden,cell = model(x,hidden,cell)
    loss += loss_func(y,y_)

  loss.backward()
  optimizer.step()

  if i % 100 == 0:
    print('\n',loss/chunk_len,'\n')
    test()
    print('\n\n')


 tensor([4.5293], grad_fn=<DivBackward0>) 

K@V/}o3q?tABIB<eJ2y9t#YG^>e6kXw$e6Pe:,E?lQGNh/H/,vJFwPel!_Q9Rsk5IG5WFV-SK[2v	'*&/k+t0UR](1Q:XZIchx=N=^=~>}f^55HpOvnlCI<BHs*` ;{u'?G@]>i`;	k



 tensor([2.5029], grad_fn=<DivBackward0>) 

brre neime nrEawigenefut p pe fennmo pon
and sth lrot poasrs say es ilariad tousou nou  wme.

r mfimh wh alirice snd chest the don he hic athe ndi, md ih thiin fort.
WH:
CL thit thirset yof save darone



 tensor([2.6200], grad_fn=<DivBackward0>) 

bed;

Thil'rou hand's
Seroo mer thernlererarn thid shel ror-de he mared ur boke he hall the ther thin songe besed the aromy the owrangh aven ime woen h congat he, sove hoescout merath, aed me hel forou



 tensor([2.4245], grad_fn=<DivBackward0>) 

bwpave ave lont souve wape doud ivece mant tow lesists you out not ended tired thar:
Iaugh thingleI the fog't for for I me herllingou yof mer al, yad astis susenrioth stersell thoe thind uyoed
unplet t



 tensor([2.2513], grad_fn=<DivBackward0>) 

bry!
Bent to to tre